In [1]:
%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
import seaborn as sns
from os.path import join
from collections import defaultdict
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
import dvu
from copy import deepcopy
import neuro.viz
import sys
from numpy import ceil
from matplotlib.colors import ListedColormap
sys.path.append('../notebooks')
from tqdm import tqdm
from neuro import config
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples
import neuro.sasc.viz
from PIL import Image
# from flatmaps_helper import VOX_COUNTS, load_custom_rois, ROI_EXPLANATIONS_S03, FED_DRIVING_EXPLANATIONS_S03, FED_DRIVING_EXPLANATIONS_S02
from neuro.flatmaps_helper import load_flatmaps, load_custom_rois, load_known_rois
import neuro.flatmaps_helper
import sys
from neuro import analyze_helper
neurosynth_compare = __import__('04_neurosynth_compare')
import neurosynth
from neuro.features.questions.gpt4 import QS_35_STABLE
import viz

/home/chansingh/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/chansingh/.venv/lib/python3.12/site-packages/cortex/mni.py:21: UserWarning: Can't find FSLDIR environment variable, assuming default FSL location..
  warnings.warn("Can't find FSLDIR environment variable, assuming default FSL location..")


In [2]:
subject = 'S02'
# subjects = [f'S0{i}' for i in range(1, 9) if not i == 6] # there's some problem with S06 surf2surf
subjects = ['S01', 'S02', 'S03', 'S08']

# load flatmaps
d = defaultdict(list)
for subject in subjects:
    # if subject in ['S01', 'S02', 'S03']:
    # settings = ['individual_gpt4',
    # 'individual_gpt4_wordrate', 'shapley_35']
    # else:
    settings = ['individual_gpt4_pc_new']
    flatmaps_qa_list = defaultdict(list)
    for setting in settings:
        flatmaps_qa_dict = joblib.load(
            join(config.PROCESSED_DIR, subject.replace('UT', ''), setting + '.pkl'))
        for q in flatmaps_qa_dict.keys():
            flatmaps_qa_list[q].append(flatmaps_qa_dict[q])
    flatmaps_qa_dict = {
        q: np.mean(flatmaps_qa_list[q], axis=0)
        for q in flatmaps_qa_list.keys()
    }
    # for k in sorted(flatmaps_qa_dict.keys()):
    for k in QS_35_STABLE:
        # print(k, flatmaps_qa_dict[k])
        # d[f'q_{subject}'].append(k)
        d[subject].append(flatmaps_qa_dict[k])

    # print(subject, len(flatmaps_qa_dict))
df = pd.DataFrame(d)
# df.set_index('q_S01', inplace=True)
df.index = QS_35_STABLE
df.index.name = 'question'
df = df[df.index.isin(QS_35_STABLE)]
assert df.shape[0] == 35

In [3]:
data = joblib.load(join(config.RESULTS_DIR_LOCAL, 'results_best_ensemble.pkl'))
rr, cols_varied, mets = data['r'], data['cols_varied'], data['mets']
metric_sort = 'corrs_tune_pc_weighted_mean'
rr_best = rr[(rr.pc_components == 100) *
             (rr.ndelays == 8) *
             (rr.feature_space.str.contains('qa_emb')) *
             (rr.feature_selection_alpha < 0) *
             (rr.num_stories == -1) *
             (rr.qa_embedding_model == 'ensemble2')
             ]  # .sort_values(by='corrs_test_mean').iloc[-1]

### Select avg-response flatmaps

In [4]:
# select
avg_defaultdict = defaultdict(list)
for subject in ['S01', 'S02', 'S03']:  # , 'S02']:
    qa_flatmaps_dict = df[subject].to_dict()

    # rois_dict = neuro.flatmaps_helper.load_known_rois(subject)
    rois_dict = neuro.flatmaps_helper.load_custom_rois(
        subject, suffix_setting='_lobes')

    corrs_35 = rr_best[rr_best.subject == subject]['corrs_test'].iloc[0]
    sig_well_predicted = corrs_35 > 0.1
    # for k in rois_dict.keys():
    # rois_dict[subject + ' lang\n(ρ>0.1)'] = sig_well_predicted & sig
    # rois_dict[subject + ' non-lang\n(ρ>0.1)'] = sig_well_predicted & ~sig

    ##################################################
    # compute means
    # avg_defaultdict = defaultdict(list)
    for i, (roi_idx, roi) in enumerate(rois_dict.items()):
        for explanation in qa_flatmaps_dict.keys():
            avg_defaultdict[roi_idx].append(
                np.mean(qa_flatmaps_dict[explanation][roi > 0]))
            if i == 0:
                avg_defaultdict['subject'].append(subject)
                avg_defaultdict['explanation'].append(explanation)
            # corrs_defaultdict[roi_idx].append(
            # np.corrcoef(roi, gemv_flatmaps_dict[explanation])[0, 1])

In [10]:
d = pd.DataFrame(avg_defaultdict)  # .set_index(['explanation'])
d = d.drop(columns='subject').groupby('explanation').mean()
d.index = [analyze_helper.abbrev_question(q) for q in d.index]
d.columns = [c.capitalize() for c in d.columns]
vabs = max(abs(d.values.min()), abs(d.values.max()))

g = sns.clustermap(d, annot=False, fmt='.3f', cmap='RdBu_r', center=0, vmin=-vabs, vmax=vabs,
               annot_kws={"size": 10}, cbar_kws={"label": "Average QA weight"}, linewidths=.5,
               )
# reverse order of rows
g.ax_heatmap.invert_yaxis()

g.ax_row_dendrogram.set_visible(False)
g.ax_col_dendrogram.set_visible(False)
g.ax_heatmap.set_xlabel('Cortical region or lobe')
cbar = g.ax_heatmap.collections[0].colorbar
cbar.remove()
plt.setp(g.ax_heatmap.get_xticklabels(), rotation=90, ha='right', fontsize=10)

# tick left
yticklabels = g.ax_heatmap.get_yticklabels()
# Remove the default left-side ticks
g.ax_heatmap.yaxis.tick_left()
g.ax_heatmap.yaxis.set_label_position('right')
for label in yticklabels:
    label.set_ha('right')
    label.set_x(-0.12)  

# assign color
q_to_idx = {analyze_helper.abbrev_question(q): i for (q,i) in viz.REMAP_QUESTIONS_TO_CATEGORY_IDXS.items()}
tab10 = plt.get_cmap('tab10')
n_colors = len(set(q_to_idx.values()))
colors = [tab10(i) for i in range(n_colors)]

for ticklabel in g.ax_heatmap.get_yticklabels():
    # ticklabel.set_color((random.random(), random.random(), random.random()))
    ticklabel.set_color(colors[q_to_idx[ticklabel.get_text()]])
joblib.dump([x.get_text() for x in g.ax_heatmap.get_yticklabels()], 'qa_rois/qa_rois_qnames_abbreviated_list.pkl')

plt.gcf().set_size_inches(4.1, 9)
# plt.show()
neuro.viz.savefig('qa_rois/qa_rois_matrix.png', dpi=400, bbox_inches='tight')

# make top-voxel fig
(based on QA-35 coefficients)

In [ ]:
subject = 'S02'
use_abs = True
if use_abs:
    vals = df[subject].abs().values
else:
    vals = df[subject].values
top_q_flatmap = np.vstack(vals).argmax(axis=0).astype(float)
corrs_35 = rr_best[rr_best.subject == subject]['corrs_test'].iloc[0]

In [ ]:
# corrs_35 = rr_best[rr_best.subject == subject]['corrs_test'].iloc[0]
# sig_well_predicted = corrs_35 > 0.1
# top_q_flatmap[~sig_well_predicted] = np.nan
# neuro.viz.quickshow(top_q_flatmap, subject=subject, cmap='tab10', center=False)

top_q_flatmap_remapped = deepcopy(top_q_flatmap)
for category_name, v in viz.REMAP_CATEGORY_TO_QUESTIONS.items():
    num_new = v[0]
    remap_nums = v[1]
    for num_old in remap_nums.keys():
        top_q_flatmap_remapped[top_q_flatmap == num_old] = num_new

# sig_well_predicted = corrs_35 > 0.15
# top_q_flatmap_remapped[~sig_well_predicted] = np.nan

n_colors = len(np.unique(top_q_flatmap_remapped[~np.isnan(top_q_flatmap_remapped)]))
tab10 = plt.get_cmap('tab10')
colors = [tab10(i) for i in range(n_colors)]
# colors = sns.color_palette("deep", as_cmap=True)
# cmap = ListedColormap(colors)
# cmap.set_bad(color='none')  # For fully transparent, use 'none'

# map top_q_flatmap_remapped into rgb array using cmap
# top_q_flatmap_rgb = cmap(top_q_flatmap_remapped / n_colors)
top_q_flatmap_rgb = [colors[int(i)] if not np.isnan(i) else (1, 1, 1, 0) for i in top_q_flatmap_remapped]
top_q_flatmap_rgb = np.array(top_q_flatmap_rgb)

alpha = corrs_35
alpha[alpha < 0] = 0  # Set negative values to 0
alpha = (alpha - alpha.min()) / (alpha.max() - alpha.min())

vol = cortex.VolumeRGB(
    top_q_flatmap_rgb[:, 0].flatten(), top_q_flatmap_rgb[:, 1].flatten(), top_q_flatmap_rgb[:, 2].flatten(),
    subject='UT' + subject, xfmname=f'UT{subject}_auto', alpha=alpha) #, cmap=cmap)

neuro.viz.quickshow(vol, with_colorbar=False,
    fname_save=f'qa_rois/qa_rois_flatmap_{subject}_abs={use_abs}.png')    

In [ ]:
# top_q_flatmap_remapped_masked = top_q_flatmap_remapped.astype(float)
# top_q_flatmap_remapped_masked[corrs_35 < 0.1] = np.nan
# neuro.viz.quickshow(
#     top_q_flatmap_remapped_masked.astype(int),
#     subject='UTS02',
#     cmap=len(np.unique(top_q_flatmap_remapped_masked)),
#     center=False,
#     with_colorbar=False,
#     fname_save='qa_rois_flatmap_masked.png',
# )

In [ ]:
max(corrs_35)

In [ ]:
# save color pallete as a colorbar
plt.figure(figsize=(10, 2))
sns.palplot(sns.color_palette("husl", 5))
plt.axis('off')
# add labels
for i, category_name in enumerate(remap.keys()):
    plt.text(i + 0.1, 0.5, category_name,
             ha='center', va='center', fontsize=12, rotation=45)
plt.savefig('qa_rois_flatmap_palette.png', dpi=300, bbox_inches='tight')